**DOWNLOAD LIBRARY**

In [2]:
import cv2
import numpy as np

**DOWNLOAD** **IMAGE**

In [4]:
img = cv2.imread("/content/pexels-andrea-piacquadio-2672979.jpg")
img= cv2.resize(img, (640, 427))

**FIND** **SHAPE**

In [ ]:
img_width = img.shape[1]
img_height = img.shape[0]

**Blob images converted to 4-dimensional tensors**.**swampRB=(BGRTORGB)**

In [ ]:
img_blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), swapRB=True, crop=False)

**YOLOv3 labels_name**

In [ ]:
labels = ["person","bicycle","car","motorcycle","airplane","bus","train","truck","boat",
           "trafficlight","firehydrant","stopsign","parkingmeter","bench","bird","cat",
           "dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack",
           "umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sportsball",
           "kite","baseballbat","baseballglove","skateboard","surfboard","tennisracket",
           "bottle","wineglass","cup","fork","knife","spoon","bowl","banana","apple",
           "sandwich","orange","broccoli","carrot","hotdog","pizza","donut","cake","chair",
           "sofa","pottedplant","bed","diningtable","toilet","tvmonitor","laptop","mouse",
           "remote","keyboard","cellphone","microwave","oven","toaster","sink","refrigerator",
           "book","clock","vase","scissors","teddybear","hairdrier","toothbrush"]

**Choose colors for labels**

In [ ]:
colors = ["0,255,250", "255,0,255", "121,120,255", "0,50,250", "70, 80, 255"]
colors = [np.array(color.split(",")).astype("int") for color in colors ]
colors = np.array(colors)
colors = np.tile(colors, (18,1))

**Cfg and Weight for model**

In [ ]:
model = cv2.dnn.readNetFromDarknet("/content/yolov3.cfg","/content/yolov3.weights")

In [ ]:
layers = model.getLayerNames()

In [ ]:
output_layer = [layers[layer-1] for layer in model.getUnconnectedOutLayers()]

In [ ]:
model.setInput(img_blob)

In [ ]:
detection_layers = model.forward(output_layer)

In [ ]:
for detection_layer in detection_layers:
    for object_detection in detection_layer:
        scores = object_detection[5:]
        predict_id = np.argmax(scores)
        confidence = scores[predict_id]
        if confidence > 0.60:
            label = labels[predict_id]
            bounding_box = object_detection[0:4] * np.array([img_width, img_height, img_width, img_height])
            (box_centre_x, box_centre_y, box_centre_witdh, box_centre_height) = bounding_box.astype("int")
            start_x = int(box_centre_x-(box_centre_witdh/2))
            start_y = int(box_centre_y - (box_centre_height/2))
            end_x = int(box_centre_x + (box_centre_witdh / 2))
            end_y = int(box_centre_y + (box_centre_height / 2))

            box_color = colors[predict_id]
            box_color = [int(each) for each in box_color]

            label = "{}: {:.2f}%".format(label, confidence*100)
            print("predicted object {}".format(label))

            cv2.rectangle(img, (start_x,start_y), (end_x,end_y),box_color,1)
            cv2.putText(img, label, (start_x, start_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color,1)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()            